In [1]:
import HardwareSetup as HS
from pydrake.all import (
    SceneGraphCollisionChecker,
    RandomGenerator,
    MosekSolver,
    GurobiSolver,
    SaveIrisRegionsYamlFile,
)
import pydrake.planning as plan

ModuleNotFoundError: No module named 'iiwa_setup'

In [2]:
diagram, context, robot, plant, internalPlantContext = HS.make_environment_model_display(hardware=False)

def get_regions():
    params = dict(edge_step_size=0.125)
    iiwa_model_instance_index = plant.GetModelInstanceByName("iiwa")
    wsg_model_instance_index = plant.GetModelInstanceByName("wsg")
    params["robot_model_instances"] = [iiwa_model_instance_index, wsg_model_instance_index]
    params["model"] = diagram
    checker = SceneGraphCollisionChecker(**params)

    options = plan.IrisFromCliqueCoverOptions()
    options.num_points_per_coverage_check = 5000
    options.num_points_per_visibility_round = 500
    options.coverage_termination_threshold = 0.7

    generator = RandomGenerator(0)

    if (MosekSolver().available() and MosekSolver().enabled()) or (
            GurobiSolver().available() and GurobiSolver().enabled()):
        # We need a MIP solver to be available to run this method.
        sets = plan.IrisInConfigurationSpaceFromCliqueCover(
            checker=checker, options=options, generator=generator,
            sets=[]
        )

        if len(sets) < 1:
            raise("No regions found")

        return sets
    else:
        print("No solvers available")

regions = get_regions()

SaveIrisRegionsYamlFile("CliqueCoverRegions.yaml", regions)

INFO:drake:Meshcat listening for connections at http://localhost:7001


RuntimeError: PackageMap::GetPackageName(): Failed to parse XML in file "/home/rohanbosworth/GCSClutterClearingDemo/iiwa_setup/scripts/../../models/package.xml".
XML_ERROR_FILE_NOT_FOUND

In [9]:
from pydrake.all import ( LoadIrisRegionsYamlFile,RandomGenerator,)
from pydrake.planning import GcsTrajectoryOptimization
from pydrake.geometry.optimization import Point, GraphOfConvexSetsOptions

edges = []
iris_regions = dict()
iris_regions.update(LoadIrisRegionsYamlFile("my_iris.yaml"))

generator = RandomGenerator()

q_start = None
q_start = iris_regions['GraspPos14'].UniformSample(generator)

while iris_regions['Between Bins'].PointInSet(q_start):
    q_start = iris_regions['GrasPosp14'].UniformSample(generator, q_start)

#Perform the same action for q_end
q_end = None
q_end = iris_regions['Above Bin 2'].UniformSample(generator)
while iris_regions['Between Bins'].PointInSet(q_end):
    q_end = iris_regions['Above Bin 2'].UniformSample(generator, q_end)



In [12]:
import time

gcs = GcsTrajectoryOptimization(len(q_start))
regions = gcs.AddRegions(list(iris_regions.values()), order=1)
source = gcs.AddRegions([Point(q_start)], order=0)
target = gcs.AddRegions([Point(q_end)], order=0)
edges.append(gcs.AddEdges(source, regions))
edges.append(gcs.AddEdges(regions, target))
        
gcs.AddPathLengthCost()
gcs.AddTimeCost(weight = 0.2)

options = GraphOfConvexSetsOptions()
options.preprocessing = True
options.max_rounded_paths = 5
start_time = time.time()
print("Running GCS")
traj, result = gcs.SolvePath(source, target, options)
print(f"GCS solved in {time.time() - start_time} seconds")

INFO:drake:Solved GCS shortest path using Clarabel with convex_relaxation=true and preprocessing=true and rounding.
INFO:drake:Finished 5 rounding trials.


Running GCS
GCS solved in 2.7190096378326416 seconds


In [6]:
import HardwareSetup as HS
from pydrake.all import (
    SceneGraphCollisionChecker,
    RandomGenerator,
    MosekSolver,
    GurobiSolver,
    SaveIrisRegionsYamlFile,
)
import pydrake.planning as plan

plant, robot_model_instances, diagram = HS.LoadRobotIRIS()

def get_regions():
    params = dict(edge_step_size=0.125)
    iiwa_model_instance_index = plant.GetModelInstanceByName("iiwa")
    wsg_model_instance_index = plant.GetModelInstanceByName("wsg")
    params["robot_model_instances"] = robot_model_instances
    params["model"] = diagram
    checker = SceneGraphCollisionChecker(**params)

    options = plan.IrisFromCliqueCoverOptions()
    options.num_points_per_coverage_check = 5000
    options.num_points_per_visibility_round = 500
    options.coverage_termination_threshold = 0.8

    generator = RandomGenerator()

    sets = plan.IrisInConfigurationSpaceFromCliqueCover(
            checker=checker, options=options, generator=generator,
            sets=[]
    )

    if len(sets) < 1:
        raise("No regions found")

    
    return sets

regions = get_regions()
regions_dict = {str(i): region for i, region in enumerate(regions)}

SaveIrisRegionsYamlFile("CliqueCoverRegions.yaml", regions_dict)

INFO:drake:Allocating contexts to support implicit context parallelism 20
INFO:drake:Using default max clique solver MaxCliqueSolverViaGreedy.
INFO:drake:Current Fraction of Domain Covered = 0
INFO:drake:IrisFromCliqueCover Iteration 1/100


file:///home/rohanbosworth/GCSClutterClearingUpdate/iiwa_setup-main/scripts/bin.sdf
/home/rohanbosworth/GCSClutterClearingUpdate/iiwa_setup-main/scripts/bin.sdf


INFO:drake:Clique added to queue. There are 146/500 points left to cover.
INFO:drake:Clique added to queue. There are 114/500 points left to cover.
INFO:drake:Clique added to queue. There are 90/500 points left to cover.
INFO:drake:Clique added to queue. There are 68/500 points left to cover.
INFO:drake:Clique added to queue. There are 41/500 points left to cover.
INFO:drake:Clique added to queue. There are 27/500 points left to cover.
INFO:drake:Clique added to queue. There are 11/500 points left to cover.
INFO:drake:Clique added to queue. There are 3/500 points left to cover.
INFO:drake:Finished adding cliques. Total of 8 clique added. Number of cliques left to process = 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurationSpace iteration 0
INFO:drake:IrisInConfigurati